<a href="https://colab.research.google.com/github/jtlagumbay/cebqa/blob/main/retriever/bm_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **QA Pipeline**

1. ElasticSeach Indexer
2. BM25 Retriever
3. Fine-tuned XLMR Reader


# Dependencies

In [1]:
!pip install pyserini faiss-cpu transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from pyserini.index import IndexWriter, IndexReader
from pyserini.search import SimpleSearcher
import json
import os
import subprocess
import time


# Indexer

In [12]:
es = Elasticsearch("https://f52c-202-92-153-104.ngrok-free.app", verify_certs=False)
print(es.info())  # Check if the connection is successful

/usr/local/lib/python3.11/dist-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://f52c-202-92-153-104.ngrok-free.app:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'f52c-202-92-153-104.ngrok-free.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


UnsupportedProductError: The client noticed that the server is not Elasticsearch and we do not support this unknown product

In [6]:
class ElasticSearchIndexer:
    def __init__(self, index_name="superbalita"):
        self.index_name = index_name
        self.es = Elasticsearch("https://7b7a-202-92-153-104.ngrok-free.app")  # Ensure ES is running

    def create_index(self):
        """ Create an index with a text field for BM25 """
        if not self.es.indices.exists(index=self.index_name):
            self.es.indices.create(index=self.index_name, body={
                "settings": {
                    "number_of_shards": 1,
                    "number_of_replicas": 0
                },
                "mappings": {
                    "properties": {
                        "id": {"type": "keyword"},
                        "title": {"type": "text"},
                        "body": {"type": "text"}
                    }
                }
            })
            print(f"Index '{self.index_name}' created.")

    def index_documents(self, documents):
        """ Bulk index documents into ElasticSearch """
        actions = [
            {
                "_index": self.index_name,
                "_id": doc["id"],  # Use document ID for uniqueness
                "_source": {
                    "id": doc["id"],
                    "title": doc["title"],
                    "body": doc["body"]
                }
            }
            for doc in documents
        ]
        bulk(self.es, actions)
        print(f"Indexed {len(documents)} documents.")



In [7]:
# Sample usage
indexer = ElasticSearchIndexer()
indexer.create_index()
documents = [
    {"id": "1", "title": "Eiffel Tower", "body": "The Eiffel Tower is in Paris."},
    {"id": "2", "title": "Python Language", "body": "Python is a programming language."},
    {"id": "3", "title": "Tokyo", "body": "The capital of Japan is Tokyo."}
]
indexer.index_documents(documents)

BadRequestError: BadRequestError(400, 'None')